In [27]:
import difflib
for i in range(1,10):
    f = open("result/0"+str(i)+".txt")
    first = f.read()
    f.close()
    f = open("gt/0"+str(i)+".txt")
    second = f.read()
    f.close()
    print('\n\n'.join(difflib.ndiff(first.splitlines(),
                                               second.splitlines())))
    print("\n-------------------------------------------------\n")

- [d1/4 e1/32 e2/2 d1/4 {e1/4,g1/4} e1/2 d1/8 c1/4 g1/16 c1/4 e1/1 ]

?                            -----


+ [ d1/4 e1/32 e2/2 d1/4 {g1/4,e1/4} e1/2 d1/8 c1/4 g1/16 c1/4 e1/1 ]

?  +                      +++++


-------------------------------------------------

  {

- [\meter<"4/4"> d1/4 e1/32 e2/2 e1/8 e1/16 e1/32 {e1/4,g1/4} e1/4 e1/8 c1/8 g1/32 c1/16 e1/32 ]

?                                                      -----        -                         -


+ [ \meter<"4/4"> d1/4 e1/32 e2/2 e1/8 e1/16 e1/32 {g1/4,e1/4} e1/4 e/8 c1/8 g1/32 c1/16 e1/32],

?  +                                                +++++                                      +


- [\meter<"4/4"> {b1/4,e1/4,g1/4} a1/8 d1/8 c1/16 g1/16 d1/16 e1/16 c2/16 g2/16 d2/16 e2/16 {a1/4,g1/4,g1/4} c1/4 a1/4. a1/8 a1/32.. ]

?               -                                                                            ^    ^                                 -


+ [ \meter<"4/4">{b1/4,e1/4,g1/4} a1/8 d1/8 c1/16 g1/16 d1/16 e1/16

0
rotation time:0.07399177551269531
binarization time:0.008640527725219727
Ref lengths time:0.07250761985778809
staff removal time:0.24378466606140137
getting lines time:0.011980772018432617
0
time: 0.49749255180358887 seconds
---------------------------------------------------------
1
rotation time:0.22142887115478516
binarization time:0.03546428680419922
Ref lengths time:0.12037801742553711
staff removal time:0.7653806209564209
getting lines time:0.0342707633972168
1
time: 1.3490991592407227 seconds
---------------------------------------------------------
2
rotation time:0.16622567176818848
binarization time:0.013484716415405273
Ref lengths time:0.07418155670166016
staff removal time:0.3027920722961426
getting lines time:0.016195297241210938
2
time: 0.6098182201385498 seconds
---------------------------------------------------------
3
rotation time:0.1161489486694336
binarization time:0.011331558227539062
Ref lengths time:0.05820775032043457
staff removal time:0.24440240859985352
ge

In [4]:
from commonfunctions import *

from scipy.signal import convolve

import segmentation
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import cv2
import matplotlib.pyplot as plt
import os 
from skimage.measure import find_contours
from skimage.color import rgb2gray , rgba2rgb
from skimage.transform import resize
from PIL import Image
import sys
import random
from sklearn.model_selection import train_test_split
# import sklearn.datasets.images as loader
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
from notes import getNoteCharacter
# import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import random
import pickle

# load the classfier
file = open("nn2.pickle",'rb')
nn = pickle.load(file)

# classifier options
target_img_size = (32, 32) # fix image size because|> classification algorithms THAT WE WILL USE HERE expect that

# We are going to fix the random seed to make our experiments reproducible 
# since some algorithms use pseudorandom generators
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)


classes = ['a_1', 'a_16', 'a_2', 'a_32', 'a_4', 'a_8', 
           'barline ', 'chord', 'clef', '.', '&&', '##', '&', '', '#', '\meter<"4/2"> ', '\meter<"4/4"> ']

dataset_dir = 'test_cases/test'#
images = load_images_from_folder(dataset_dir)

fileNames = os.listdir(dataset_dir)

for i,img in enumerate(images):
    print(i)
    start_time = time.time()
    start = time.time()
    rotated = rotateImage(img)
    print('rotation time:' + str(time.time() - start))

    start = time.time()
    binary = binraization(rotated)//255
    print('binarization time:' + str(time.time() - start))
    
    # show_images([binary])

    start = time.time()
    staffHeight, spaceHeight = getRefLengths(binary)
    print('Ref lengths time:' + str(time.time() - start))
    
    start = time.time()
    filteredImg, candidates = getCandidateStaffs(binary, staffHeight)
    filteredImg, candidates, eliminated = removeLonelyStaffs(candidates, binary, staffHeight, spaceHeight, eliminated=[])
#     filteredImg1, candidates, eliminated = RemoveThinStaffs(candidates, filteredImg, staffHeight)
#     filteredImg2, candidates, eliminated = removeLonelyStaffs(candidates, binary, staffHeight, spaceHeight, eliminated)
#     filteredImg3, candidates = addFalseNegatives(candidates, filteredImg2, staffHeight, staffHeight, eliminated)
#     # print(filteredImg3)
    staffLess = (binary-filteredImg).astype(np.uint8)
    print('staff removal time:' + str(time.time() - start))

    start = time.time()
    lines = getLines(1-filteredImg, staffHeight, spaceHeight)
    staff = cv2.cvtColor(staffLess, cv2.COLOR_GRAY2BGR)
    staff[lines] = [0,0,255]
    plt.imsave('staff'  +'.png', staff)
    print('getting lines time:' + str(time.time() - start))
    
    objects = segmentImage(staffLess, lines, staffHeight, spaceHeight)
    firstTime = True
    output = ""
    perviousAccedental = ""
    for o in objects:
        features = extract_hog_features(staffLess[o[1]:o[3], o[0]:o[2]],target_img_size)
        symbol_name = classes[np.argmax( nn.predict_proba([features]))]

#         show_images([staffLess[o[1]:o[3], o[0]:o[2]]])
        if symbol_name == "a_2" or symbol_name == "a_4":
            if isHalf(staffLess[o[1]:o[3], o[0]:o[2]],spaceHeight) :
                symbol_name = "a_2"
            else:
                symbol_name = "a_4"
        if symbol_name =="clef":
            if firstTime:
                firstTime = False
                output+= '['
            else:
                output+= ']\n['
        elif firstTime:
            continue 
        #beam
        elif (o[2]-o[0]) > 4*spaceHeight:
            try:
                output += getNoteCharacter(staffLess, o, "beam", lines, staffHeight, spaceHeight)+" "
            except:
                continue
        #dot and barline
        elif symbol_name == "." or symbol_name == "barline ":
                if isDot(staffLess[o[1]:o[3], o[0]:o[2]],spaceHeight):
                    output += "."
        
        #chord
        elif symbol_name == "chord":
#             print("chord")
            try:
                notes = getchordText([o[1],o[3]],staffLess[o[1]:o[3], o[0]:o[2]],staffHeight,spaceHeight,lines)
            except:
                noteSymbol = getNoteCharacter(staffLess, o, "a_4", lines, staffHeight, spaceHeight)
       
                # print(noteSymbol)
                output += noteSymbol[0]+perviousAccedental+noteSymbol[1:]+" "
                perviousAccedental = ""
                # print('contin')

                continue
            output +="{"
            for k in range(0,len(notes)-2,2):
                output += notes[k:k+2]+"/4,"
            output += notes[-2:]+"/4"
            output+= "} "
            
        #note
        elif symbol_name!="" and  symbol_name[0]=="a" :
            try:
                noteSymbol = getNoteCharacter(staffLess, o, symbol_name, lines, staffHeight, spaceHeight)
                output += noteSymbol[0]+perviousAccedental+noteSymbol[1:]+" "
                perviousAccedental = ""
            except:
                # print('contin')
                continue
        #accedentals
        elif symbol_name == '\meter<"4/2"> ' or symbol_name == '\meter<"4/4"> ':
            output += symbol_name
        else:

            perviousAccedental= symbol_name
    output+="]"
    print(i)
    if len(output.split("\n"))>1:
        output ="{\n"+output+"\n}"
    f =open('result'+"/"+fileNames[i].split(".")[0]+".txt",'w') 
    f.write(output)
    f.close()
    print("time: %s seconds" % (time.time() - start_time))
    print("---------------------------------------------------------")




0
rotation time:0.07100224494934082
binarization time:0.014499187469482422
Ref lengths time:0.0780789852142334
staff removal time:0.2669847011566162
getting lines time:0.012510299682617188
0
time: 0.4783003330230713 seconds
---------------------------------------------------------
1
rotation time:0.25023674964904785
binarization time:0.03977346420288086
Ref lengths time:0.12248587608337402
staff removal time:0.7988617420196533
getting lines time:0.03654289245605469
1
time: 1.3697888851165771 seconds
---------------------------------------------------------
2
rotation time:0.16580414772033691
binarization time:0.017720699310302734
Ref lengths time:0.0713338851928711
staff removal time:0.30135655403137207
getting lines time:0.015846967697143555
2
time: 0.655076265335083 seconds
---------------------------------------------------------
3
rotation time:0.11498546600341797
binarization time:0.012909412384033203
Ref lengths time:0.04876518249511719
staff removal time:0.2573516368865967
getti